In [52]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime
%reload_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


In [53]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


final_pt_df2 = pd.read_csv(Path(wd + '/data/raw/csv/01102018_final_pt_df2_v.csv') , index_col=0) #only for patients with minimum vitals
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 199 ms


In [95]:
#adding this in fow now, will eventually want to switch this to the 03-clinical_variables_final
import psycopg2
import getpass
conn = psycopg2.connect(dbname="mimic", user="student", host="luolabwkstn01.fsm.northwestern.edu", port="5432",
                        password=getpass.getpass(prompt='Password:'.format()))
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Password: ·····


time: 3.16 s


In [76]:
#reading in all of my data that is not limited to 72 hour time window between t_0 and t+72
date= '27082018' 

ventcategory_df=pd.read_csv(Path(
    wd+'/data/raw/csv/%s_ventcategory_df.csv' %(date)), index_col=0)

vaso_dose_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_vaso_dose_72.csv' %(date)), index_col=0)


echodata_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_echodata_72.csv' %(date)), index_col=0)

labs_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_labs_all_nosummary_72.csv' %(date)), index_col=0)


weightfirstday_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_weightfirstday_df.csv' %(date)), index_col=0)
heightfirstday_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_heightfirstday_df.csv' %(date)), index_col=0)

vitals_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_vitals_all_nosummary_72.csv' %(date)), index_col=0)
uti_all_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_uti_all_72.csv' %(date)), index_col=0)
bg_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_bg_all_nosummary_72.csv' %(date)), index_col=0)

rrt_merged_allpt_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_rrt_merged_allpt_df.csv' %(date)), index_col=0)
gcs72_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_gcs72_df.csv' %(date)), index_col=0)

sofa_df_72=pd.read_csv(Path(
    wd+'/data/raw/csv/%s_sofa_df_72.csv' %(date)), index_col=0)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 14.9 s


In [77]:
def df_subset(df):
    try:
        df = df.loc[df.loc[:,'icustay_id'].isin(icustay_id),:]
    except KeyError or NameError:
        try:
            df = df.loc[df.loc[:,'hadm_id'].isin(hadm_id),:]
        except KeyError or NameError:
            df = df.loc[df.loc[:,'subject_id'].isin(patients),:]
    return(df)

time: 4.73 ms


In [78]:
# subsetting each dataframe to only patients in final_patients_df2_v:
dataframe_list= [
    ventcategory_df, vaso_dose_72, echodata_72,
    labs_all_nosummary_72, weightfirstday_df,
    heightfirstday_df, vitals_all_nosummary_72,
    uti_all_72, bg_all_nosummary_72,
    rrt_merged_allpt_df, gcs72_df, sofa_df_72
]

(
ventcategory_df, vaso_dose_72, echodata_72,
labs_all_nosummary_72, weightfirstday_df,
heightfirstday_df, vitals_all_nosummary_72,
uti_all_72, bg_all_nosummary_72,
rrt_merged_allpt_df, gcs72_df, sofa_df_72
) = (df_subset(df) for df in dataframe_list)

time: 1.03 s


## vaso dose

In [79]:
#vaso_dose_72.head()
vaso_dose_72= vaso_dose_72.rename(
    columns={'starttime':'charttime','label':'vaso_type'}
        )#.drop(labels='endtime', axis=1) 
#renaming starttime to charttime and dropping endtime

time: 43.5 ms


In [80]:
len(vaso_dose_72)

53786

time: 3.66 ms


In [81]:
vaso_dose_72.head()

,icustay_id,charttime,endtime,vaso_rate,vaso_amount,amount_uom,rate_uom,vaso_type,t_0,delta
4,200349.0,2139-06-02 16:31:00,2139-06-02 18:10:00,0.012030,0.073958,mg,mcg/kg/min,epinephrine,2139-06-02,0 days 16:31:00.000000000
92,200952.0,2139-09-23 15:07:00,2139-09-24 06:43:00,0.020004,1.049601,mg,mcg/kg/min,epinephrine,2139-09-23,0 days 15:07:00.000000000
93,200952.0,2139-09-25 12:00:00,2139-09-25 21:59:00,0.080068,1.962759,mg,mcg/kg/min,epinephrine,2139-09-23,2 days 12:00:00.000000000
94,200952.0,2139-09-25 21:59:00,2139-09-26 07:38:00,0.060601,2.000000,mg,mcg/kg/min,epinephrine,2139-09-23,2 days 21:59:00.000000000
106,200952.0,2139-09-23 23:12:00,2139-09-24 21:54:00,0.149992,7.990853,mg,mcg/kg/min,norepinephrine,2139-09-23,0 days 23:12:00.000000000


time: 32.2 ms


In [82]:
#removing units/hour because that is a different use of vasopressin
vaso_dose_72 = vaso_dose_72.loc[vaso_dose_72.loc[:,'rate_uom']!= 'units/hour',:]
vaso_dose_72 = vaso_dose_72.loc[vaso_dose_72.loc[:,'rate_uom']!= 'Uhr',:]

time: 36.6 ms


In [83]:
len(vaso_dose_72)

52976

time: 2.79 ms


In [84]:
vaso_dose_72['rate_uom'].unique()

array(['mcg/kg/min', 'mcgkgmin', 'Umin'], dtype=object)

time: 4.51 ms


#### removing outliers

In [86]:
# Use transform to add a column back to the orig df from a groupby aggregation, transform returns a Series with its index aligned to the orig df:

test_group=(vaso_dose_72[['vaso_rate','rate_uom','amount_uom','vaso_type']]#.groupby('vaso_type', as_index=False)
     .groupby(['vaso_type','rate_uom'])
         )

vaso_dose_72['std']=test_group.transform(lambda x : x.std())
vaso_dose_72['mean']=test_group.transform(lambda x : x.mean())

normal_high_value= pd.DataFrame({
    'vaso_type' : ['dobutamine','dopamine','epinephrine','norepinephrine','vasopressin','phenylephrine'],
    'high_value': [40, 20, 0.5, 1, 0.1, 2]
}) #found from literature, see notes

vaso_dose_72 = pd.merge(vaso_dose_72, normal_high_value, left_on='vaso_type', right_on='vaso_type')

vaso_dose_72_rmout =(vaso_dose_72[
    ~((vaso_dose_72['vaso_rate'] > vaso_dose_72['high_value']) & ((vaso_dose_72['vaso_rate']-vaso_dose_72['mean'])>= (3*vaso_dose_72['std'])))
    ])
#ie vaso_dose_72_rmout is a dataframe of all rows that excludes rows where vaso rate > literature high value and where vaso_rate >3sd from teh mean

time: 142 ms


In [88]:
len(vaso_dose_72) #52976

52976

time: 2.23 ms


In [89]:
vaso_dose_72['vaso_type'].unique()

array(['epinephrine', 'norepinephrine', 'phenylephrine', 'vasopressin',
       'dopamine', 'dobutamine'], dtype=object)

time: 4.09 ms


In [90]:
print(len(vaso_dose_72_rmout), len(vaso_dose_72)) #works, only excluded ~300/52662 for outliers. 


52662 52976
time: 1.7 ms


In [34]:
vaso_dose_72_rmout['rate_uom'].unique()

array(['mcg/kg/min', 'mcgkgmin', 'Umin'], dtype=object)

time: 3.07 ms


In [144]:
date= '22102018'
pd.DataFrame(vaso_dose_72_rmout).to_csv(Path(
    wd+'/data/processed/%s_vaso_dose_72_rmout.csv' %(date)))


time: 652 ms


## vitals

In [20]:
vitals_all_nosummary_72 = vitals_all_nosummary_72.rename(index=str, columns={"valueuom":"uom"}) #change valueom to uom
vitals_all_nosummary_nonull = vitals_all_nosummary_72.loc[vitals_all_nosummary_72['vitalid'].notnull(),:]#.count() #6930 NULL values

vitals_all_nosummary_nonull.loc[vitals_all_nosummary_nonull.loc[:,'uom']=='BPM','uom']='bpm'

#overall the values are extremely similar and are likely the same thing
#i will combine them.
vitals_all_nosummary_nonull.loc[
    (vitals_all_nosummary_nonull['vitalid']=='RespRate') & 
    (vitals_all_nosummary_nonull['uom']=='bpm'),'uom']='insp/min'

vitals_all_nosummary_nonull.loc[
    (vitals_all_nosummary_nonull['vitalid']=='TempC') & 
    (vitals_all_nosummary_nonull['uom']=='?C'),'uom']='Deg. C'

vitals_all_nosummary_nonull.loc[
    (vitals_all_nosummary_nonull['vitalid']=='TempF') & 
    (vitals_all_nosummary_nonull['uom']=='Deg. F'),'uom']='Deg. C'

vitals_all_nosummary_nonull.loc[
    (vitals_all_nosummary_nonull['vitalid']=='TempF') & 
    (vitals_all_nosummary_nonull['uom']=='?F'),'uom']='Deg. C'


vitals_all_nosummary_nonull.loc[
    (vitals_all_nosummary_nonull['vitalid']=='TempF'),'vitalid']='TempC'

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


time: 12.6 s


In [21]:
vitals_all_nosummary_nonull.head()

,subject_id,hadm_id,icustay_id,charttime,uom,vitalid,valuenum,t_0,delta
5,9,150750,220597,2149-11-13 15:20:00,mmHg,MeanArtPress,61.0,2149-11-13,0 days 15:20:00.000000000
6,9,150750,220597,2149-11-14 11:00:00,mmHg,MeanArtPress,103.0,2149-11-13,1 days 11:00:00.000000000
7,9,150750,220597,2149-11-14 12:00:00,mmHg,MeanArtPress,134.0,2149-11-13,1 days 12:00:00.000000000
12,9,150750,220597,2149-11-13 07:00:00,mmHg,MeanArtPress,85.0,2149-11-13,0 days 07:00:00.000000000
13,9,150750,220597,2149-11-14 02:00:00,mmHg,MeanArtPress,96.0,2149-11-13,1 days 02:00:00.000000000
16,9,150750,220597,2149-11-13 10:00:00,mmHg,MeanArtPress,92.0,2149-11-13,0 days 10:00:00.000000000
18,9,150750,220597,2149-11-13 00:30:00,mmHg,MeanArtPress,104.0,2149-11-13,0 days 00:30:00.000000000
19,9,150750,220597,2149-11-13 17:00:00,mmHg,MeanArtPress,84.0,2149-11-13,0 days 17:00:00.000000000
29,9,150750,220597,2149-11-13 18:00:00,mmHg,MeanArtPress,93.0,2149-11-13,0 days 18:00:00.000000000
46,9,150750,220597,2149-11-13 04:00:00,mmHg,MeanArtPress,104.0,2149-11-13,0 days 04:00:00.000000000


time: 32.7 ms


In [92]:
vitals_all_nosummary_nonull.groupby('vitalid')['uom'].value_counts() #looks good

vitalid       uom     
DiasBP        mmHg         920800
Glucose       mg/dL         38696
HeartRate     bpm          955108
MeanArtPress  mmHg         933617
RespRate      insp/min    1043762
SpO2          %            929932
SysBP         mmHg         921226
TempC         Deg. C       255419
Name: uom, dtype: int64

time: 1.62 s


In [93]:
pd.DataFrame(vitals_all_nosummary_nonull).to_csv(Path(
    wd+'/data/processed/%s_vitals_all_nosummary_nonull.csv' %(date)))


time: 41.1 s


# labs

In [96]:
#importing unit of mesurements:
def uom_sql_import(file_path):
    if isinstance(file_path, str)== True:
        f = open(Path(file_path), 'r')
    else:
        f = open(Path(str(file_path)), 'r')

    SQL = open(file_path,'r').read()
    SQL_df= pd.read_sql_query(SQL,conn)   
    return(SQL_df)

time: 7.04 ms


In [97]:
lab_uom= uom_sql_import(Path(wd+'/data/raw/clinical_var_sql/unit_of_mesurement/labs_uom.sql'))
labs_all_nosummary_72 = pd.merge(labs_all_nosummary_72, lab_uom, left_on='label', right_on='label')
del(lab_uom)

time: 22.1 s


In [99]:
labs_all_nosummary_72.groupby('label')['uom'].value_counts() #looks good

label        uom    
ALBUMIN      g/dL       10756
ANION GAP    mEq/L      61650
BANDS        %           3516
BICARBONATE  mEq/L      62606
BILIRUBIN    mg/dL      17814
BUN          mg/dL      63051
CHLORIDE     mEq/L      68812
CREATININE   mg/dL      63315
GLUCOSE      mg/dL      81925
HEMATOCRIT   %          78508
HEMOGLOBIN   g/dL       64940
LACTATE      mmol/L     34490
PLATELET     K/uL       58060
POTASSIUM    mEq/L      86455
PT           SECONDS    39677
PTT          sec        41760
SODIUM       mEq/L      71894
WBC          K/uL       56390
Name: uom, dtype: int64

time: 485 ms


In [ ]:
#outliers

In [104]:
pd.DataFrame(labs_all_nosummary_72).to_csv(Path(
    wd+'/data/processed/%s_labs_all_nosummary_72.csv' %(date)))

time: 7.32 s


## vent category

In [101]:
ventcategory_df['uom']='category'
ventcategory_df['dtype']= 'factor'
ventcategory_df.head()

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,icustay_id,t_0,vent_day1,vent_day2,vent_day3,uom,dtype
0,267059.0,2182-11-12,Mech,None,None,category,factor
1,278729.0,2159-11-08,Oxygen,Mech,None,category,factor
2,219670.0,2190-08-01,Oxygen,None,None,category,factor
3,295185.0,2154-11-11,Oxygen,None,None,category,factor
4,211472.0,2193-12-11,None,None,None,category,factor


time: 137 ms


In [103]:
pd.DataFrame(ventcategory_df).to_csv(Path(
    wd+'/data/processed/%s_ventcategory_df.csv' %(date)))

time: 67.5 ms


## weight and height firstday
i explored weightdurations and it had more missing values than weightfirstday, so i will use that. we can revisit this if we need longitudinal weights

In [105]:
weightfirstday_df['uom']='kg'

time: 53.9 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [106]:
weightfirstday_df

,icustay_id,weight,weight_admit,weight_daily,weight_echoinhosp,weight_echoprehosp,uom
7,200012,51.20,51.200001,51.200001,NaN,NaN,kg
8,200014,62.00,62.000000,NaN,NaN,NaN,kg
10,200019,65.00,65.000000,NaN,NaN,NaN,kg
16,200030,113.60,113.599998,113.599998,NaN,NaN,kg
18,200033,74.00,74.000000,NaN,NaN,NaN,kg
21,200036,79.00,NaN,79.000000,NaN,NaN,kg
33,200055,56.00,56.000000,NaN,NaN,NaN,kg
34,200059,54.00,54.000000,60.000000,NaN,NaN,kg
35,200061,250.00,250.000000,NaN,NaN,NaN,kg
37,200063,NaN,NaN,NaN,NaN,NaN,kg


time: 25.4 ms


In [107]:
pd.DataFrame(weightfirstday_df).to_csv(Path(
    wd+'/data/processed/%s_weightfirstday_df.csv' %(date)))

time: 112 ms


In [108]:
heightfirstday_df['uom']='cm'

time: 48.4 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [109]:
pd.DataFrame(heightfirstday_df).to_csv(Path(
    wd+'/data/processed/%s_heightfirstday_df.csv' %(date)))

time: 84.4 ms


# UTI

In [123]:
uti_all_72_notnull= uti_all_72.loc[uti_all_72.loc[:,'value'].notnull(),:] #has uom attached, but is irrelevant because all null
uti_all_72_notnull= uti_all_72_notnull.loc[uti_all_72_notnull.loc[:,'value']!=' ',:]

time: 104 ms


In [124]:
uti_all_72_notnull.head()

,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,label,fluid,category,loinc_code,icd9_code,t_0,delta
60,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,99831,2131-05-21,0 days 01:33:00.000000000
61,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,99811,2131-05-21,0 days 01:33:00.000000000
62,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,41511,2131-05-21,0 days 01:33:00.000000000
63,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,4538,2131-05-21,0 days 01:33:00.000000000
64,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,99672,2131-05-21,0 days 01:33:00.000000000


time: 16 ms


In [125]:
uti_all_72_notnull['label'].unique()

array(['Leukocytes', 'Nitrite'], dtype=object)

time: 7.64 ms


In [126]:
uti_all_72_notnull['category'].unique()

array(['Hematology'], dtype=object)

time: 8.77 ms


In [128]:
uti_all_72_notnull['value'].unique()#seems good

array(['NEG', 'MOD', 'TR', 'POS', 'SM', 'LG', 'SMALL'], dtype=object)

time: 6.84 ms


In [129]:
pd.DataFrame(uti_all_72_notnull).to_csv(Path(
    wd+'/data/processed/%s_uti_all_72_notnull.csv' %(date)))

time: 1.87 s


# bloodgas

In [ ]:
bg_all_nosummary_72= bg_all_nosummary_72.loc[bg_all_nosummary_72['value'].notnull(),:]
bg_all_nosummary_72 = bg_all_nosummary_72.rename(index=str, columns={'valueuom':'uom'})
bg_all_nosummary_72= bg_all_nosummary_72.loc[~(bg_all_nosummary_72.loc[:,'value']=='.'),:] #removing value= ' .'

#may need to remove outliers, haven't done as of 10/22/18
bg_all_nosummary_72.head()

In [130]:
pd.DataFrame(bg_all_nosummary_72).to_csv(Path(
    wd+'/data/processed/%s_bg_all_nosummary_72.csv' %(date)))

time: 5.63 s


In [ ]:
#may need to 

# RRT

In [132]:
rrt_merged_allpt_df_notnull = rrt_merged_allpt_df.loc[rrt_merged_allpt_df['rrt'].notnull(),:]
rrt_merged_allpt_df_notnull['uom']='category'
rrt_merged_allpt_df_notnull.head()

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,icustay_id,t_0,first_charttime,rrt,uom
7,252135.0,2166-04-13,2166-04-13 00:00:00,1.0,category
8,293255.0,2167-10-21,2167-10-21 04:31:00,1.0,category
9,261279.0,2167-10-26,2167-10-26 01:00:00,1.0,category
20,251526.0,2129-12-20,2129-12-20 09:32:00,1.0,category
31,255314.0,2181-10-11,2181-10-11 01:00:00,1.0,category


time: 390 ms


In [133]:
pd.DataFrame(rrt_merged_allpt_df_notnull).to_csv(Path(
    wd+'/data/processed/%s_rrt_merged_allpt_df_notnull.csv' %(date)))

time: 9.56 ms


# GCS_72

In [136]:
gcs72_df['uom']='GCS_score'

time: 48.7 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [137]:
pd.DataFrame(gcs72_df).to_csv(Path(
    wd+'/data/processed/%s_gcs72_df.csv' %(date)))

time: 148 ms


# SOFA_72
i'm going to remove all sofa variables except daily score, as we have other markers for those in our above data
i may later use this as qc check.

In [140]:
sofa_score_72= sofa_df_72[['subject_id','hadm_id','icustay_id','day','sofa','t_0']]

time: 6.81 ms


In [142]:
sofa_score_72['uom']='daily_sofa_score'

time: 58.3 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [143]:
pd.DataFrame(sofa_score_72).to_csv(Path(
    wd+'/data/processed/%s_sofa_score_72.csv' %(date)))

time: 220 ms


# echo data

In [146]:
echodata_72.head()

,row_id,subject_id,hadm_id,chartdate,charttime,indication,height,weight,bsa,bp,bpsys,bpdias,hr,status,test,doppler,contrast,technicalquality,t_0,delta
0,59653,31038,174978.0,2102-06-14 00:00:00,2102-06-14 13:30:00,Endocarditis.,74.0,165.0,2.00,102/72,102.0,72.0,86.0,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate,2102-06-13,1 days 13:30:00.000000000
1,59654,70150,156140.0,2191-05-10 00:00:00,2191-05-10 15:08:00,"CHF, A-FIB",66.0,263.0,2.25,107/81,107.0,81.0,95.0,Outpatient,TTE (Complete),Full Doppler and color Doppler,None,Suboptimal,2191-05-10,0 days 15:08:00.000000000
2,59655,54190,188571.0,2158-02-16 00:00:00,2158-02-16 13:48:00,Cerebrovascular event/TIA. Streptococcal bacte...,72.0,187.0,2.07,147/87,147.0,87.0,90.0,Inpatient,Portable TEE (Complete),Full Doppler and color Doppler,None,Adequate,2158-02-15,1 days 13:48:00.000000000
3,59657,80030,100442.0,2119-06-07 00:00:00,2119-06-07 14:59:00,Left ventricular function.,63.0,126.0,1.59,88/53,88.0,53.0,115.0,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate,2119-06-07,0 days 14:59:00.000000000
4,59659,82208,188268.0,2200-10-22 00:00:00,2200-10-22 15:14:00,"Acute MR worsening,ESRD, on HD, NSLS lung Ca, ...",59.0,136.0,1.57,126/61,126.0,61.0,95.0,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate,2200-10-22,0 days 15:14:00.000000000


time: 24.2 ms


In [147]:
#null values:
pd.DataFrame(
    {
    'bsa':[echodata_72.loc[echodata_72['bsa'].isna(),'hadm_id'].count()], 
    'bp': [echodata_72.loc[echodata_72['bp'].isna(),'hadm_id'].count()], 
    'bpsys': [echodata_72.loc[echodata_72['bpsys'].isna(),'hadm_id'].count()], 
    'bpdias': [echodata_72.loc[echodata_72['bpdias'].isna(),'hadm_id'].count()],
    'hr': [echodata_72.loc[echodata_72['hr'].isna(),'hadm_id'].count()],
    'test': [echodata_72.loc[echodata_72['test'].isna(),'hadm_id'].count()]
             }
            )

,bp,bpdias,bpsys,bsa,hr,test
0,506,518,518,1088,700,0


time: 68.7 ms


In [148]:
pd.DataFrame(
    {
    'bsa':[echodata_72.loc[echodata_72['bsa'].notna(),'hadm_id'].count()], 
    'bp': [echodata_72.loc[echodata_72['bp'].notna(),'hadm_id'].count()], 
    'bpsys': [echodata_72.loc[echodata_72['bpsys'].notna(),'hadm_id'].count()], 
    'bpdias': [echodata_72.loc[echodata_72['bpdias'].notna(),'hadm_id'].count()],
    'hr': [echodata_72.loc[echodata_72['hr'].notna(),'hadm_id'].count()],
    'test': [echodata_72.loc[echodata_72['test'].notna(),'hadm_id'].count()]
             }
            )

,bp,bpdias,bpsys,bsa,hr,test
0,5438,5426,5426,4856,5244,5944


time: 20.4 ms


In [150]:
echodata_72.loc[echodata_72.loc[:,'doppler']=='No doppler','doppler']='No Doppler'
echodata_72.loc[echodata_72.loc[:,'doppler']=='Color doppler only','doppler']='Color Doppler only'
echodata_72.loc[echodata_72.loc[:,'doppler']=='Full doppler and color doppler','doppler']='Full Doppler and color Doppler'
echodata_72.loc[echodata_72.loc[:,'doppler']=='Limited doppler and no color doppler','doppler']='Limited Doppler and no color Doppler'
echodata_72.loc[echodata_72.loc[:,'doppler']=='Limited doppler and color doppler','doppler']='Limited Doppler and color Doppler'

echodata_72['doppler'].unique()

time: 284 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [152]:
echodata_72['doppler'].unique()

array(['Full Doppler and color Doppler', 'Complete pulse and color flow',
       'Limited Doppler and color Doppler',
       'Limited Doppler and no color Doppler', 'Color Doppler only',
       'Focused pulse and color flow', 'No Doppler'], dtype=object)

time: 4.38 ms


In [155]:
echodata_72['doppler'].value_counts()

Full Doppler and color Doppler          4716
Complete pulse and color flow            478
Limited Doppler and color Doppler        445
No Doppler                               125
Focused pulse and color flow              66
Limited Doppler and no color Doppler      62
Color Doppler only                        52
Name: doppler, dtype: int64

time: 4.73 ms


In [158]:
echodata_72_df= echodata_72[['subject_id','hadm_id','charttime','bsa','bp','bpsys','bpdias','test','doppler']]

time: 3.42 ms


In [159]:
pd.DataFrame(echodata_72_df).to_csv(Path(
    wd+'/data/processed/%s_echodata_72_df.csv' %(date)))

time: 58.8 ms
